In [177]:
# Import required libraries
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set pandas display options to show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 20)




## Raw Data Into DataFrames

In [178]:
# Define file paths - adjust this path to match your project location
# Replace this with the actual path to your work_supa project
project_path = Path(r"C:\Users\Eddy\YTM Capital Dropbox\Eddy Winiarz\Trading\COF\Models\Unfinished Models\Eddy\Python Projects\work_supa")

parquet_files = {
    'universe': project_path / 'universe' / 'universe.parquet',
    'portfolio': project_path / 'portfolio' / 'portfolio.parquet',
    'combined_runs': project_path / 'runs' / 'combined_runs.parquet',
    'run_monitor': project_path / 'runs' / 'run_monitor.parquet',
    'bond_z': project_path / 'historical g spread' / 'bond_z.parquet'
}

print("📊 Pipeline Parquet Files to DataFrames")
print("=" * 50)
print("🔍 Checking parquet file availability:")
for name, path in parquet_files.items():
    status = "✅" if path.exists() else "❌"
    print(f"{status} {name}: {path}")
print()

# Load all parquet files into DataFrames
print("📥 Loading parquet files into DataFrames...")
print("=" * 50)

# Dictionary to store all DataFrames
dataframes = {}

for name, file_path in parquet_files.items():
    if file_path.exists():
        try:
            # Load the parquet file
            df = pd.read_parquet(file_path)
            
            # Store in dictionary
            dataframes[name] = df
            
            # Also create a variable with the same name as the file
            globals()[name] = df
            
            print(f"✅ {name}: {df.shape[0]:,} rows × {df.shape[1]} columns ({file_path.stat().st_size / 1024 / 1024:.1f} MB)")
            
        except Exception as e:
            print(f"❌ Error loading {name}: {e}")
    else:
        print(f"❌ File not found: {file_path}")

print(f"\n📊 Total DataFrames loaded: {len(dataframes)}")

# Simple .info() display for all DataFrames
print("📊 DATAFRAME INFO SUMMARY")
print("=" * 80)


# Simple .info() for all DataFrames
print("📊 DATAFRAME INFO")
print("=" * 80)

for name, df in dataframes.items():
    print(f"\n {name.upper()}")
    print("=" * 40)
    df.info()
    print()

📊 Pipeline Parquet Files to DataFrames
🔍 Checking parquet file availability:
✅ universe: C:\Users\Eddy\YTM Capital Dropbox\Eddy Winiarz\Trading\COF\Models\Unfinished Models\Eddy\Python Projects\work_supa\universe\universe.parquet
✅ portfolio: C:\Users\Eddy\YTM Capital Dropbox\Eddy Winiarz\Trading\COF\Models\Unfinished Models\Eddy\Python Projects\work_supa\portfolio\portfolio.parquet
✅ combined_runs: C:\Users\Eddy\YTM Capital Dropbox\Eddy Winiarz\Trading\COF\Models\Unfinished Models\Eddy\Python Projects\work_supa\runs\combined_runs.parquet
✅ run_monitor: C:\Users\Eddy\YTM Capital Dropbox\Eddy Winiarz\Trading\COF\Models\Unfinished Models\Eddy\Python Projects\work_supa\runs\run_monitor.parquet
✅ bond_z: C:\Users\Eddy\YTM Capital Dropbox\Eddy Winiarz\Trading\COF\Models\Unfinished Models\Eddy\Python Projects\work_supa\historical g spread\bond_z.parquet

📥 Loading parquet files into DataFrames...
✅ universe: 38,917 rows × 47 columns (4.9 MB)
✅ portfolio: 9,390 rows × 51 columns (1.2 MB)
✅ co

## Creating The z_all_enhanced DataFrame

In [179]:
# Create z_all_enhanced DataFrame - OPTIMIZED VERSION
print("Creating z_all_enhanced DataFrame (OPTIMIZED)...")
print("=" * 60)

import time
start_time = time.time()

# Start with all columns from BOND_Z
z_all_enhanced = dataframes['bond_z'].copy()

# Get the last date from universe and portfolio
last_universe_date = dataframes['universe']['Date'].max()
last_portfolio_date = dataframes['portfolio']['Date'].max()

print(f"Last universe date: {last_universe_date}")
print(f"Last portfolio date: {last_portfolio_date}")

# Define UNIVERSE columns to join
universe_columns = ['Custom_Sector',
    'CPN TYPE', 'Ticker', 'G Sprd', 'Yrs (Cvn)', 'Currency', 
    'Equity Ticker', 'Yrs Since Issue', 'Risk', 'Rating', 
    'Yrs Since Issue Bucket', 'Yrs (Mat) Bucket'
]

# Define RUN_MONITOR columns to join
run_monitor_columns = [
    'Bid/Offer', 'Dealer @ Best Bid', 'Dealer @ Best Offer', 
    'Size @ Best Bid', 'Size @ Best Offer'
]

print(f"⚡ Starting with {len(z_all_enhanced):,} rows from BOND_Z")

# OPTIMIZATION 1: Pre-filter UNIVERSE and RUN_MONITOR to only needed columns and CUSIPs
print("\nPre-filtering data for faster joins...")

# Get unique CUSIPs from BOND_Z
bond_cusips = set(z_all_enhanced['CUSIP_1'].unique()) | set(z_all_enhanced['CUSIP_2'].unique())

# Filter UNIVERSE to only needed CUSIPs and columns, and only last date
universe_filtered = dataframes['universe'][
    (dataframes['universe']['CUSIP'].isin(bond_cusips)) &
    (dataframes['universe']['Date'] == last_universe_date)
][['CUSIP'] + universe_columns].copy()

# Filter RUN_MONITOR to only needed CUSIPs and columns  
run_monitor_filtered = dataframes['run_monitor'][
    dataframes['run_monitor']['CUSIP'].isin(bond_cusips)
][['CUSIP'] + run_monitor_columns].copy()

# Filter PORTFOLIO to only last date
portfolio_filtered = dataframes['portfolio'][
    dataframes['portfolio']['Date'] == last_portfolio_date
].copy()

print(f"✅ UNIVERSE filtered: {len(universe_filtered):,} rows (from {len(dataframes['universe']):,})")
print(f"✅ RUN_MONITOR filtered: {len(run_monitor_filtered):,} rows (from {len(dataframes['run_monitor']):,})")
print(f"✅ PORTFOLIO filtered: {len(portfolio_filtered):,} rows (from {len(dataframes['portfolio']):,})")

# OPTIMIZATION 2: Create lookup dictionaries for faster joins
print("\n🔧 Creating lookup dictionaries...")

# UNIVERSE lookup dictionaries
universe_lookup_1 = {}
universe_lookup_2 = {}
for _, row in universe_filtered.iterrows():
    cusip = row['CUSIP']
    universe_lookup_1[cusip] = {f"{col}_1": row[col] for col in universe_columns}
    universe_lookup_2[cusip] = {f"{col}_2": row[col] for col in universe_columns}

# RUN_MONITOR lookup dictionaries
run_monitor_lookup_1 = {}
run_monitor_lookup_2 = {}
for _, row in run_monitor_filtered.iterrows():
    cusip = row['CUSIP']
    run_monitor_lookup_1[cusip] = {f"{col}_1": row[col] for col in run_monitor_columns}
    run_monitor_lookup_2[cusip] = {f"{col}_2": row[col] for col in run_monitor_columns}

# PORTFOLIO CUSIP set for fast membership testing (only last date)
portfolio_cusips = set(portfolio_filtered['CUSIP'].unique())

print(f"✅ Lookup dictionaries created for {len(bond_cusips):,} unique CUSIPs")

# OPTIMIZATION 3: Vectorized operations for all joins
print("\n⚡ Performing vectorized joins...")

# Add UNIVERSE data for CUSIP_1
for col in universe_columns:
    z_all_enhanced[f"{col}_1"] = z_all_enhanced['CUSIP_1'].map(
        {k: v[f"{col}_1"] for k, v in universe_lookup_1.items()}
    )

# Add UNIVERSE data for CUSIP_2  
for col in universe_columns:
    z_all_enhanced[f"{col}_2"] = z_all_enhanced['CUSIP_2'].map(
        {k: v[f"{col}_2"] for k, v in universe_lookup_2.items()}
    )

# Add RUN_MONITOR data for CUSIP_1
for col in run_monitor_columns:
    z_all_enhanced[f"{col}_1"] = z_all_enhanced['CUSIP_1'].map(
        {k: v[f"{col}_1"] for k, v in run_monitor_lookup_1.items()}
    )

# Add RUN_MONITOR data for CUSIP_2
for col in run_monitor_columns:
    z_all_enhanced[f"{col}_2"] = z_all_enhanced['CUSIP_2'].map(
        {k: v[f"{col}_2"] for k, v in run_monitor_lookup_2.items()}
    )

# Add ownership columns (vectorized) - only for last portfolio date
z_all_enhanced['Own_CUSIP_1'] = z_all_enhanced['CUSIP_1'].isin(portfolio_cusips).astype(int)
z_all_enhanced['Own_CUSIP_2'] = z_all_enhanced['CUSIP_2'].isin(portfolio_cusips).astype(int)

# Performance summary
end_time = time.time()
execution_time = end_time - start_time

print("\n" + "=" * 60)
print("🎉 z_all_enhanced DataFrame created successfully!")
print(f"⚡ Execution time: {execution_time:.3f} seconds")
print(f"📊 Final shape: {z_all_enhanced.shape[0]:,} rows × {z_all_enhanced.shape[1]} columns")

# Quick validation
print("\nVALIDATION SUMMARY:")
print("=" * 30)
print(f"✅ CUSIP_1 UNIVERSE matches: {z_all_enhanced['Ticker_1'].notna().sum():,}/{len(z_all_enhanced):,}")
print(f"✅ CUSIP_2 UNIVERSE matches: {z_all_enhanced['Ticker_2'].notna().sum():,}/{len(z_all_enhanced):,}")
print(f"✅ CUSIP_1 RUN_MONITOR matches: {z_all_enhanced['Bid/Offer_1'].notna().sum():,}/{len(z_all_enhanced):,}")
print(f"✅ CUSIP_2 RUN_MONITOR matches: {z_all_enhanced['Bid/Offer_2'].notna().sum():,}/{len(z_all_enhanced):,}")
print(f"✅ Own_CUSIP_1 positions: {z_all_enhanced['Own_CUSIP_1'].sum():,}")
print(f"✅ Own_CUSIP_2 positions: {z_all_enhanced['Own_CUSIP_2'].sum():,}")

print(f"\n🚀 Performance: {len(z_all_enhanced):,} records processed in {execution_time:.3f}s")
print(f"⚡ Speed: {len(z_all_enhanced)/execution_time:,.0f} records/second")

Creating z_all_enhanced DataFrame (OPTIMIZED)...
Last universe date: 2025-07-16 00:00:00
Last portfolio date: 2025-06-30 00:00:00
⚡ Starting with 19,900 rows from BOND_Z

Pre-filtering data for faster joins...
✅ UNIVERSE filtered: 182 rows (from 38,917)
✅ RUN_MONITOR filtered: 519 rows (from 5,830)
✅ PORTFOLIO filtered: 654 rows (from 9,390)

🔧 Creating lookup dictionaries...
✅ Lookup dictionaries created for 200 unique CUSIPs

⚡ Performing vectorized joins...

🎉 z_all_enhanced DataFrame created successfully!
⚡ Execution time: 0.072 seconds
📊 Final shape: 19,900 rows × 47 columns

VALIDATION SUMMARY:
✅ CUSIP_1 UNIVERSE matches: 18,385/19,900
✅ CUSIP_2 UNIVERSE matches: 17,833/19,900
✅ CUSIP_1 RUN_MONITOR matches: 10,856/19,900
✅ CUSIP_2 RUN_MONITOR matches: 8,845/19,900
✅ Own_CUSIP_1 positions: 515
✅ Own_CUSIP_2 positions: 281

🚀 Performance: 19,900 records processed in 0.072s
⚡ Speed: 277,999 records/second


## Edits To z_all_enhanced

In [180]:
# Create z_all_enhanced DataFrame - OPTIMIZED VERSION (CLEAN)
# Start with all columns from BOND_Z
z_all_enhanced = dataframes['bond_z'].copy()

# Get the last date from universe and portfolio
last_universe_date = dataframes['universe']['Date'].max()
last_portfolio_date = dataframes['portfolio']['Date'].max()

print(f"Last universe date: {last_universe_date}")
print(f"Last portfolio date: {last_portfolio_date}")

# Define UNIVERSE columns to join
universe_columns = ['Custom_Sector',
    'CPN TYPE', 'Ticker', 'G Sprd', 'Yrs (Cvn)', 'Currency', 
    'Equity Ticker', 'Yrs Since Issue', 'Risk', 'Rating', 
    'Yrs Since Issue Bucket', 'Yrs (Mat) Bucket'
]

# Define RUN_MONITOR columns to join
run_monitor_columns = [
    'Bid/Offer', 'Dealer @ Best Bid', 'Dealer @ Best Offer', 
    'Size @ Best Bid', 'Size @ Best Offer'
]

# Get unique CUSIPs from BOND_Z
bond_cusips = set(z_all_enhanced['CUSIP_1'].unique()) | set(z_all_enhanced['CUSIP_2'].unique())

# Filter UNIVERSE to only needed CUSIPs and columns, and only last date
universe_filtered = dataframes['universe'][
    (dataframes['universe']['CUSIP'].isin(bond_cusips)) &
    (dataframes['universe']['Date'] == last_universe_date)
][['CUSIP'] + universe_columns].copy()

# Filter RUN_MONITOR to only needed CUSIPs and columns  
run_monitor_filtered = dataframes['run_monitor'][
    dataframes['run_monitor']['CUSIP'].isin(bond_cusips)
][['CUSIP'] + run_monitor_columns].copy()

# Filter PORTFOLIO to only last date
portfolio_filtered = dataframes['portfolio'][
    dataframes['portfolio']['Date'] == last_portfolio_date
].copy()

# Create lookup dictionaries for faster joins
universe_lookup_1 = {}
universe_lookup_2 = {}
for _, row in universe_filtered.iterrows():
    cusip = row['CUSIP']
    universe_lookup_1[cusip] = {f"{col}_1": row[col] for col in universe_columns}
    universe_lookup_2[cusip] = {f"{col}_2": row[col] for col in universe_columns}

run_monitor_lookup_1 = {}
run_monitor_lookup_2 = {}
for _, row in run_monitor_filtered.iterrows():
    cusip = row['CUSIP']
    run_monitor_lookup_1[cusip] = {f"{col}_1": row[col] for col in run_monitor_columns}
    run_monitor_lookup_2[cusip] = {f"{col}_2": row[col] for col in run_monitor_columns}

# PORTFOLIO CUSIP set for fast membership testing (only last date)
portfolio_cusips = set(portfolio_filtered['CUSIP'].unique())

# Add UNIVERSE data for CUSIP_1
for col in universe_columns:
    z_all_enhanced[f"{col}_1"] = z_all_enhanced['CUSIP_1'].map(
        {k: v[f"{col}_1"] for k, v in universe_lookup_1.items()}
    )

# Add UNIVERSE data for CUSIP_2  
for col in universe_columns:
    z_all_enhanced[f"{col}_2"] = z_all_enhanced['CUSIP_2'].map(
        {k: v[f"{col}_2"] for k, v in universe_lookup_2.items()}
    )

# Add RUN_MONITOR data for CUSIP_1
for col in run_monitor_columns:
    z_all_enhanced[f"{col}_1"] = z_all_enhanced['CUSIP_1'].map(
        {k: v[f"{col}_1"] for k, v in run_monitor_lookup_1.items()}
    )

# Add RUN_MONITOR data for CUSIP_2
for col in run_monitor_columns:
    z_all_enhanced[f"{col}_2"] = z_all_enhanced['CUSIP_2'].map(
        {k: v[f"{col}_2"] for k, v in run_monitor_lookup_2.items()}
    )

# Add ownership columns (vectorized) - only for last portfolio date
z_all_enhanced['Own_CUSIP_1'] = z_all_enhanced['CUSIP_1'].isin(portfolio_cusips).astype(int)
z_all_enhanced['Own_CUSIP_2'] = z_all_enhanced['CUSIP_2'].isin(portfolio_cusips).astype(int)

# Delete specified columns
columns_to_delete = [
    'Dealer @ Best Bid_1',
    'Size @ Best Bid_1', 
    'Dealer @ Best Offer_2',
    'Size @ Best Offer_2'
]
z_all_enhanced = z_all_enhanced.drop(columns=columns_to_delete)

# Add CR01 columns
risk_1_numeric = pd.to_numeric(z_all_enhanced['Risk_1'], errors='coerce')
risk_2_numeric = pd.to_numeric(z_all_enhanced['Risk_2'], errors='coerce')

z_all_enhanced['CUSIP_1 CR01 @ Wide Offer'] = (
    (z_all_enhanced['Size @ Best Offer_1'].fillna(0) / 1000000) * risk_1_numeric.fillna(0)
)

z_all_enhanced['CUSIP_2 CR01 @ Best Bid'] = (
    (z_all_enhanced['Size @ Best Bid_2'].fillna(0) / 1000000) * risk_2_numeric.fillna(0)
)

# Move CUSIP_1 and CUSIP_2 to the end
cols_without_cusips = [col for col in z_all_enhanced.columns if col not in ['CUSIP_1', 'CUSIP_2']]
z_all_enhanced = z_all_enhanced[cols_without_cusips + ['CUSIP_1', 'CUSIP_2']]

# Sort by Z_Score high to low (descending)
z_all_enhanced = z_all_enhanced.sort_values('Z_Score', ascending=False).reset_index(drop=True)

# Define numeric columns to round to 1 decimal (excluding CR01 and Size columns)
numeric_cols_1_decimal = [
    'Last_Spread', 'Z_Score', 'Max', 'Min', 'Last_vs_Max', 'Last_vs_Min', 'Percentile',
    'G Sprd_1', 'Yrs (Cvn)_1', 'Yrs Since Issue_1', 'G Sprd_2', 'Yrs (Cvn)_2', 'Yrs Since Issue_2',
    'Bid/Offer_1', 'Bid/Offer_2'
]

# Apply formatting within display() only
def format_for_display(df):
    """Format DataFrame for display without changing underlying data"""
    # Set display options to show full content
    pd.set_option('display.max_colwidth', None)  # Show full column content
    pd.set_option('display.width', None)  # Don't wrap wide tables
    pd.set_option('display.max_columns', None)  # Show all columns
    pd.set_option('display.max_rows', None)  # Show all rows
    
    formatted_df = df.copy()
    
    # Round numeric columns to 1 decimal
    for col in numeric_cols_1_decimal:
        if col in formatted_df.columns:
            formatted_df[col] = formatted_df[col].round(1)
    
    # Format CR01 columns with thousand separators and no decimals
    formatted_df['CUSIP_1 CR01 @ Wide Offer'] = formatted_df['CUSIP_1 CR01 @ Wide Offer'].apply(
        lambda x: f"{x:,.0f}" if pd.notna(x) else ""
    )
    formatted_df['CUSIP_2 CR01 @ Best Bid'] = formatted_df['CUSIP_2 CR01 @ Best Bid'].apply(
        lambda x: f"{x:,.0f}" if pd.notna(x) else ""
    )
    
    # Format Size columns with thousand separators and no decimals
    formatted_df['Size @ Best Offer_1'] = formatted_df['Size @ Best Offer_1'].apply(
        lambda x: f"{x:,.0f}" if pd.notna(x) else ""
    )
    formatted_df['Size @ Best Bid_2'] = formatted_df['Size @ Best Bid_2'].apply(
        lambda x: f"{x:,.0f}" if pd.notna(x) else ""
    )
    
    # Format Risk columns with thousand separators and no decimals
    formatted_df['Risk_1'] = formatted_df['Risk_1'].apply(
        lambda x: f"{x:,.0f}" if pd.notna(x) else ""
    )
    formatted_df['Risk_2'] = formatted_df['Risk_2'].apply(
        lambda x: f"{x:,.0f}" if pd.notna(x) else ""
    )
    
    return formatted_df

display(format_for_display(z_all_enhanced.head(20)))

Last universe date: 2025-07-16 00:00:00
Last portfolio date: 2025-06-30 00:00:00


,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
0,AQNCN 4.6 01/29/29,ATIMLP 6.069 02/05/42,-84.4,4.2,-82.3,-146.0,-2.1,61.6,99.4,Power Gen/Renewable,FIXED,AQNCN,NaN,4.0,CAD,AQN CN Equity,6.5,,NR,5-7,3.1-4.1,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,NaN,NaN,,NaN,NaN,,0,0,0,0,01585PAJ4,04682BAA0
1,AERMON 3.441 04/26/51,ATIMLP 6.069 02/05/42,-53.1,4.1,-51.1,-100.2,-2.1,47.0,99.4,Airport,FIXED,AERMON,102.2,25.8,CAD,58232Z CN Equity,4.2,"1,248",A+,3-5,>25.1,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,6.0,RBC,"5,000,000",NaN,NaN,,0,0,"6,241",0,007863AM5,04682BAA0
2,AERMON 3.03 04/21/50,ATIMLP 6.069 02/05/42,-52.1,4.1,-49.9,-99.2,-2.1,47.2,99.4,Airport,FIXED,AERMON,103.3,24.8,CAD,58232Z CN Equity,5.2,"1,170",A+,5-7,10.1-25.1,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,6.0,RBC,"5,000,000",NaN,NaN,,0,0,"5,851",0,007863AL7,04682BAA0
3,AERMON 3.36 04/24/47,ATIMLP 6.069 02/05/42,-47.8,4.0,-45.4,-96.6,-2.4,48.8,99.4,Airport,FIXED,AERMON,107.9,21.8,CAD,58232Z CN Equity,8.2,"1,153",A+,>7,10.1-25.1,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,5.0,NBF,"3,000,000",NaN,NaN,,0,0,"3,460",0,007863AK9,04682BAA0
4,AERMON 3.918 06/12/45,ATIMLP 6.069 02/05/42,-44.7,4.0,-42.5,-94.6,-2.2,49.9,99.4,Airport,FIXED,AERMON,110.8,19.9,CAD,58232Z CN Equity,10.1,"1,166",A+,>7,10.1-25.1,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,5.0,NBF,"3,000,000",NaN,NaN,,0,0,"3,499",0,007863AJ2,04682BAA0
5,AQNCN 4.09 02/17/27,ATIMLP 6.069 02/05/42,-92.5,4.0,-90.4,-154.1,-2.1,61.6,99.4,Power Gen/Renewable,FIXED,AQNCN,NaN,2.0,CAD,AQN CN Equity,8.5,,NR,>7,1-2.1,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,NaN,NaN,,NaN,NaN,,0,0,0,0,01585PAG0,04682BAA0
6,ALTALK 2.747 05/29/26,ATIMLP 6.069 02/05/42,-131.5,3.9,-129.3,-188.1,-2.2,56.6,99.4,Utility,FIXED,ALTALK,24.0,0.9,CAD,BRK/A US Equity,9.1,85,A-,>7,0.50-1,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,7.0,RBC,"10,000,000",NaN,NaN,,0,0,854,0,02135ZAR7,04682BAA0
7,APUCN 3.636 04/21/25,ATIMLP 6.069 02/05/42,-88.3,3.9,-86.2,-148.3,-2.1,60.0,99.4,REIT,FIXED,APUCN,NaN,0.2,CAD,AP-U CN Equity,8.2,,NR,>7,0-0.25,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,NaN,NaN,,NaN,NaN,,0,0,0,0,019456AC6,04682BAA0
8,AAPL 0.55 08/20/25,ATIMLP 6.069 02/05/42,-100.7,3.9,-100.7,-206.6,0.0,105.9,99.8,Non Financial Maples,FIXED,AAPL,33.6,0.1,USD,AAPL US Equity,4.9,10,AA+,3-5,0-0.25,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,NaN,NaN,,NaN,NaN,,0,0,0,0,037833DX5,04682BAA0
9,ACICN 4.26 12/05/28,ATIMLP 6.069 02/05/42,-77.9,3.8,-73.9,-124.3,-4.0,46.4,99.0,Utility,FIXED,ACICN,79.4,3.1,CAD,ACI CN Equity,6.6,298,BBB+,5-7,3.1-4.1,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,5.0,TD,"3,000,000",NaN,NaN,,0,0,894,0,02091ZAA6,04682BAA0


In [181]:
z_all_enhanced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19900 entries, 0 to 19899
Data columns (total 45 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Security_1                 19900 non-null  object 
 1   Security_2                 19900 non-null  object 
 2   Last_Spread                19900 non-null  float64
 3   Z_Score                    19900 non-null  float64
 4   Max                        19900 non-null  float64
 5   Min                        19900 non-null  float64
 6   Last_vs_Max                19900 non-null  float64
 7   Last_vs_Min                19900 non-null  float64
 8   Percentile                 19900 non-null  float64
 9   Custom_Sector_1            18385 non-null  object 
 10  CPN TYPE_1                 18385 non-null  object 
 11  Ticker_1                   18385 non-null  object 
 12  G Sprd_1                   17527 non-null  float64
 13  Yrs (Cvn)_1                18385 non-null  flo

In [ ]:
# All CAD Bonds IN the Universe

# Create z_cad_universe DataFrame - only CAD currency pairs
z_cad_universe = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'CAD')
].copy()

display(format_for_display(z_cad_universe.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
0,AQNCN 4.6 01/29/29,ATIMLP 6.069 02/05/42,-84.4,4.2,-82.3,-146.0,-2.1,61.6,99.4,Power Gen/Renewable,FIXED,AQNCN,NaN,4.0,CAD,AQN CN Equity,6.5,,NR,5-7,3.1-4.1,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,NaN,NaN,,NaN,NaN,,0,0,0,0,01585PAJ4,04682BAA0
1,AERMON 3.441 04/26/51,ATIMLP 6.069 02/05/42,-53.1,4.1,-51.1,-100.2,-2.1,47.0,99.4,Airport,FIXED,AERMON,102.2,25.8,CAD,58232Z CN Equity,4.2,"1,248",A+,3-5,>25.1,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,6.0,RBC,"5,000,000",NaN,NaN,,0,0,"6,241",0,007863AM5,04682BAA0
2,AERMON 3.03 04/21/50,ATIMLP 6.069 02/05/42,-52.1,4.1,-49.9,-99.2,-2.1,47.2,99.4,Airport,FIXED,AERMON,103.3,24.8,CAD,58232Z CN Equity,5.2,"1,170",A+,5-7,10.1-25.1,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,6.0,RBC,"5,000,000",NaN,NaN,,0,0,"5,851",0,007863AL7,04682BAA0
3,AERMON 3.36 04/24/47,ATIMLP 6.069 02/05/42,-47.8,4.0,-45.4,-96.6,-2.4,48.8,99.4,Airport,FIXED,AERMON,107.9,21.8,CAD,58232Z CN Equity,8.2,"1,153",A+,>7,10.1-25.1,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,5.0,NBF,"3,000,000",NaN,NaN,,0,0,"3,460",0,007863AK9,04682BAA0
4,AERMON 3.918 06/12/45,ATIMLP 6.069 02/05/42,-44.7,4.0,-42.5,-94.6,-2.2,49.9,99.4,Airport,FIXED,AERMON,110.8,19.9,CAD,58232Z CN Equity,10.1,"1,166",A+,>7,10.1-25.1,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,5.0,NBF,"3,000,000",NaN,NaN,,0,0,"3,499",0,007863AJ2,04682BAA0
5,AQNCN 4.09 02/17/27,ATIMLP 6.069 02/05/42,-92.5,4.0,-90.4,-154.1,-2.1,61.6,99.4,Power Gen/Renewable,FIXED,AQNCN,NaN,2.0,CAD,AQN CN Equity,8.5,,NR,>7,1-2.1,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,NaN,NaN,,NaN,NaN,,0,0,0,0,01585PAG0,04682BAA0
6,ALTALK 2.747 05/29/26,ATIMLP 6.069 02/05/42,-131.5,3.9,-129.3,-188.1,-2.2,56.6,99.4,Utility,FIXED,ALTALK,24.0,0.9,CAD,BRK/A US Equity,9.1,85,A-,>7,0.50-1,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,7.0,RBC,"10,000,000",NaN,NaN,,0,0,854,0,02135ZAR7,04682BAA0
7,APUCN 3.636 04/21/25,ATIMLP 6.069 02/05/42,-88.3,3.9,-86.2,-148.3,-2.1,60.0,99.4,REIT,FIXED,APUCN,NaN,0.2,CAD,AP-U CN Equity,8.2,,NR,>7,0-0.25,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,NaN,NaN,,NaN,NaN,,0,0,0,0,019456AC6,04682BAA0
9,ACICN 4.26 12/05/28,ATIMLP 6.069 02/05/42,-77.9,3.8,-73.9,-124.3,-4.0,46.4,99.0,Utility,FIXED,ACICN,79.4,3.1,CAD,ACI CN Equity,6.6,298,BBB+,5-7,3.1-4.1,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,5.0,TD,"3,000,000",NaN,NaN,,0,0,894,0,02091ZAA6,04682BAA0
12,ATH 5.113 03/07/29,ATIMLP 6.069 02/05/42,-65.6,3.6,-56.7,-117.3,-8.9,51.7,99.0,Financial Maples,FIXED,ATH,90.1,3.6,CAD,ATH US Equity,1.4,346,A+,1-2,3.1-4.1,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,5.0,TD,"3,000,000",NaN,NaN,,0,0,"1,038",0,04686PAD9,04682BAA0


In [ ]:
# Comparing Same Term

z_term = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['Yrs (Mat) Bucket_1'] == z_all_enhanced['Yrs (Mat) Bucket_2'])
].copy()

display(format_for_display(z_term.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
2,AERMON 3.03 04/21/50,ATIMLP 6.069 02/05/42,-52.1,4.1,-49.9,-99.2,-2.1,47.2,99.4,Airport,FIXED,AERMON,103.3,24.8,CAD,58232Z CN Equity,5.2,"1,170",A+,5-7,10.1-25.1,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,6.0,RBC,"5,000,000",NaN,NaN,,0,0,"5,851",0,007863AL7,04682BAA0
3,AERMON 3.36 04/24/47,ATIMLP 6.069 02/05/42,-47.8,4.0,-45.4,-96.6,-2.4,48.8,99.4,Airport,FIXED,AERMON,107.9,21.8,CAD,58232Z CN Equity,8.2,"1,153",A+,>7,10.1-25.1,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,5.0,NBF,"3,000,000",NaN,NaN,,0,0,"3,460",0,007863AK9,04682BAA0
4,AERMON 3.918 06/12/45,ATIMLP 6.069 02/05/42,-44.7,4.0,-42.5,-94.6,-2.2,49.9,99.4,Airport,FIXED,AERMON,110.8,19.9,CAD,58232Z CN Equity,10.1,"1,166",A+,>7,10.1-25.1,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,5.0,NBF,"3,000,000",NaN,NaN,,0,0,"3,499",0,007863AJ2,04682BAA0
13,ALTALK 4.054 11/21/44,ATIMLP 6.069 02/05/42,-38.9,3.6,-37.8,-89.0,-1.1,50.0,99.0,Utility,FIXED,ALTALK,114.7,19.4,CAD,BRK/A US Equity,10.6,"1,156",A-,>7,10.1-25.1,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,1.0,NBF,"3,000,000",NaN,NaN,,0,0,"3,468",0,02135ZAP1,04682BAA0
14,AERMON 3.919 09/26/42,ATIMLP 6.069 02/05/42,-41.9,3.6,-39.1,-93.0,-2.8,51.1,99.0,Airport,FIXED,AERMON,114.2,17.2,CAD,58232Z CN Equity,12.8,"1,076",A+,>7,10.1-25.1,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,5.0,NBF,"3,000,000",NaN,NaN,,0,0,"3,227",0,007863AH6,04682BAA0
17,ALTALK 4.09 06/30/45,ATIMLP 6.069 02/05/42,-41.0,3.6,-38.9,-90.0,-2.1,49.0,99.0,Utility,FIXED,ALTALK,114.4,20.0,CAD,BRK/A US Equity,10.0,"1,179",A-,>7,10.1-25.1,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,1.0,NBF,"5,000,000",NaN,NaN,,0,0,"5,894",0,02135ZAQ9,04682BAA0
26,ALCTRA 3.458 04/12/49,ATIMLP 6.069 02/05/42,-46.9,3.5,-44.1,-92.0,-2.8,45.1,99.0,Utility,FIXED,ALCTRA,109.2,23.7,CAD,1487407D CN Equity,6.3,"1,200",A-,5-7,10.1-25.1,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,4.0,BMO,"2,000,000",NaN,NaN,,0,0,"2,399",0,014443AK1,04682BAA0
28,AERMON 5.472 04/16/40,ATIMLP 6.069 02/05/42,-36.9,3.5,-34.7,-90.3,-2.2,53.4,99.4,Airport,FIXED,AERMON,118.6,14.8,CAD,58232Z CN Equity,15.2,"1,088",A+,>7,10.1-25.1,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,NaN,None,,NaN,NaN,,0,0,0,0,007863AG8,04682BAA0
31,ALTALK 3.717 12/03/46,ATIMLP 6.069 02/05/42,-43.0,3.5,-41.2,-94.2,-1.7,51.3,99.0,Utility,FIXED,ALTALK,112.0,21.4,CAD,BRK/A US Equity,8.6,"1,177",A-,>7,10.1-25.1,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,2.0,NBF,"3,000,000",NaN,NaN,,0,0,"3,531",0,02135ZAS5,04682BAA0
34,ALCTRA 5.297 04/29/41,ATIMLP 6.069 02/05/42,-34.4,3.5,-32.4,-87.1,-2.0,52.7,99.4,Utility,FIXED,ALCTRA,120.9,15.8,CAD,1487407D CN Equity,14.2,"1,121",A-,>7,10.1-25.1,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,NaN,None,,NaN,NaN,,0,0,0,0,014443AF2,04682BAA0


In [198]:
# Comparing Same Sector

z_sector = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['Custom_Sector_1'] == z_all_enhanced['Custom_Sector_2'])
].copy()

display(format_for_display(z_sector.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
112,AIREIN 3.367 06/01/27,BCIMCR 2.551 06/24/26,27.9,3.1,27.9,10.5,0.0,17.4,99.8,Pension,FIXED,AIREIN,43.8,1.7,CAD,1351035D CN Equity,6.6,166,NR,5-7,1-2.1,Pension,FIXED,BCIMCR,16.0,0.9,CAD,1790650D CN Equity,3.4,92,NR,3-5,0.50-1,2.0,TD,"3,000,000",NaN,CIBC,"10,000,000",0,0,497,921,00889YAC5,05554PAE3
180,AIREIN 2.195 11/04/26,BCIMCR 2.551 06/24/26,19.5,3.0,19.8,-0.8,-0.3,20.4,99.4,Pension,FIXED,AIREIN,35.7,1.3,CAD,1351035D CN Equity,3.7,126,NR,3-5,1-2.1,Pension,FIXED,BCIMCR,16.0,0.9,CAD,1790650D CN Equity,3.4,92,NR,3-5,0.50-1,5.0,TD,"3,000,000",NaN,CIBC,"10,000,000",0,0,378,921,00889YAE1,05554PAE3
199,AIREIN 4.97 05/23/34,BCIMCR 2.551 06/24/26,83.4,2.9,83.6,69.0,-0.2,14.4,99.0,Pension,FIXED,AIREIN,99.6,8.6,CAD,1351035D CN Equity,1.1,726,NR,1-2,7.1-10.1,Pension,FIXED,BCIMCR,16.0,0.9,CAD,1790650D CN Equity,3.4,92,NR,3-5,0.50-1,4.0,SCM,"3,000,000",NaN,CIBC,"10,000,000",0,0,"2,178",921,00889YAG6,05554PAE3
269,ALACN 4 ½ 08/15/44,ALACN 4.99 10/04/47,10.7,2.7,13.1,-12.6,-2.4,23.3,99.4,Midstream,FIXED,ALACN,179.0,19.1,CAD,ALA CN Equity,10.9,"1,077",BBB-,>7,10.1-25.1,Midstream,FIXED,ALACN,169.5,22.2,CAD,ALA CN Equity,7.8,"1,222",BBB-,>7,10.1-25.1,NaN,None,,NaN,SCM,"3,000,000",0,0,0,"3,667",02138ZAL7,02138ZAR4
273,ACOXCN 5 ½ 11/01/2078,ALACN 8.9 11/10/2083,-34.8,2.7,-33.7,-150.3,-1.1,115.5,99.4,Non Financial Hybrids,VARIABLE,ACOXCN,178.7,3.3,CAD,ACO/X CN Equity,6.7,308,BBB-,5-7,>25.1,Non Financial Hybrids,VARIABLE,ALACN,212.4,3.3,CAD,ALA CN Equity,1.7,324,BB,1-2,>25.1,NaN,NaN,,16.0,CIBC,"2,000,000",0,0,0,649,046789AA1,021361AC4
293,NWHYDR 3.877 12/31/36,TVECN 7 ¼ 05/10/27,-45.6,2.6,-44.1,-252.6,-1.5,206.9,99.4,HY,FIXED,NWHYDR,190.0,11.5,CAD,0192104D CN Equity,6.3,771,NR,5-7,10.1-25.1,HY,FIXED,TVECN,234.1,0.8,CAD,TVE CN Equity,2.8,81,nan,2-3,1-2.1,NaN,NaN,,NaN,NaN,,0,0,0,0,66815PAB6,87505YAD4
297,ALTALK 4.872 11/15/40,ALTALK 4.922 09/17/43,7.3,2.6,7.6,-1.5,-0.3,8.7,98.2,Utility,FIXED,ALTALK,121.4,15.3,CAD,BRK/A US Equity,14.7,"1,067",A-,>7,10.1-25.1,Utility,FIXED,ALTALK,116.7,18.2,CAD,BRK/A US Equity,11.8,"1,196",A-,>7,10.1-25.1,2.0,NBF,"2,000,000",5.0,TD,"3,000,000",0,0,"2,134","3,588",02135ZAD8,02135ZAK2
345,BCECN 3.8 08/21/28,BCECN 4 ¾ 09/29/44,-88.8,2.5,-88.8,-108.2,0.0,19.4,99.8,Cable/Telco,FIXED,BCECN,73.9,2.9,CAD,BCE CN Equity,6.9,270,BBB,5-7,3.1-4.1,Cable/Telco,FIXED,BCECN,163.8,19.2,CAD,BCE CN Equity,10.8,"1,129",BBB,>7,10.1-25.1,2.0,RBC,"5,000,000",5.0,TD,"5,000,000",0,0,"1,348","5,643",07813ZCA0,07813ZBH6
387,ALTALK 4.462 11/08/41,ALTALK 4.922 09/17/43,5.0,2.5,6.1,-1.3,-1.2,6.2,95.8,Utility,FIXED,ALTALK,119.3,16.3,CAD,BRK/A US Equity,13.7,"1,080",A-,>7,10.1-25.1,Utility,FIXED,ALTALK,116.7,18.2,CAD,BRK/A US Equity,11.8,"1,196",A-,>7,10.1-25.1,1.0,NBF,"2,000,000",5.0,TD,"3,000,000",0,0,"2,159","3,588",02135ZAE6,02135ZAK2
412,BCECN 3.55 03/02/26,BCECN 5.85 11/10/32,-64.4,2.4,-63.3,-82.6,-1.1,18.2,97.8,Cable/Telco,FIXED,BCECN,49.2,0.4,CAD,BCE CN Equity,9.4,38,BBB,>7,0.50-1,Cable/Telco,FIXED,BCECN,112.6,7.1,CAD,BCE CN Equity,2.7,632,BBB,2-3,7.1-10.1,5.0,SCM,"10,171,000",4.0,NBF,"10,000,000",0,0,386,"6,319",07813ZBT0,07813ZCK8


In [199]:
# Comparing Same Ticker

z_ticker = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['Ticker_1'] == z_all_enhanced['Ticker_2'])
].copy()

display(format_for_display(z_ticker.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
220,ALACN 5.141 03/14/34,ALACN 8.9 11/10/2083,-87.1,2.9,-86.1,-198.5,-1.0,111.5,99.4,Midstream,FIXED,ALACN,126.4,8.4,CAD,ALA CN Equity,1.3,702,BBB-,1-2,7.1-10.1,Non Financial Hybrids,VARIABLE,ALACN,212.4,3.3,CAD,ALA CN Equity,1.7,324,BB,1-2,>25.1,5.0,RBC,"2,000,000",16.0,CIBC,"2,000,000",0,0,"1,404",649,02138ZBJ1,021361AC4
226,ALACN 4.12 04/07/26,ALACN 8.9 11/10/2083,-157.7,2.8,-156.1,-270.5,-1.6,112.8,99.4,Midstream,FIXED,ALACN,56.3,0.5,CAD,ALA CN Equity,9.3,48,BBB-,>7,0.50-1,Non Financial Hybrids,VARIABLE,ALACN,212.4,3.3,CAD,ALA CN Equity,1.7,324,BB,1-2,>25.1,5.0,SCM,"3,925,000",16.0,CIBC,"2,000,000",1,0,187,649,02138ZAP8,021361AC4
233,ALACN 2.477 11/30/30,ALACN 8.9 11/10/2083,-123.1,2.8,-123.1,-229.0,0.0,106.0,99.8,Midstream,FIXED,ALACN,87.7,5.4,CAD,ALA CN Equity,4.6,459,BBB-,3-5,5.1-7.1,Non Financial Hybrids,VARIABLE,ALACN,212.4,3.3,CAD,ALA CN Equity,1.7,324,BB,1-2,>25.1,-2.0,BMO,"5,000,000",16.0,CIBC,"2,000,000",0,0,"2,293",649,02138ZAX1,021361AC4
257,ALACN 4 ½ 08/15/44,ALACN 8.9 11/10/2083,-33.9,2.7,-33.5,-144.9,-0.5,110.9,99.4,Midstream,FIXED,ALACN,179.0,19.1,CAD,ALA CN Equity,10.9,"1,077",BBB-,>7,10.1-25.1,Non Financial Hybrids,VARIABLE,ALACN,212.4,3.3,CAD,ALA CN Equity,1.7,324,BB,1-2,>25.1,NaN,None,,16.0,CIBC,"2,000,000",0,0,0,649,02138ZAL7,021361AC4
265,ALACN 4.638 05/15/26,ALACN 8.9 11/10/2083,-159.6,2.7,-159.6,-272.7,0.0,113.1,99.8,Midstream,FIXED,ALACN,50.6,0.8,CAD,ALA CN Equity,2.2,83,BBB-,2-3,0.50-1,Non Financial Hybrids,VARIABLE,ALACN,212.4,3.3,CAD,ALA CN Equity,1.7,324,BB,1-2,>25.1,10.0,NBF,"5,000,000",16.0,CIBC,"2,000,000",1,0,413,649,02138ZBG7,021361AC4
267,ALACN 3.98 10/04/27,ALACN 8.9 11/10/2083,-146.5,2.7,-145.4,-258.2,-1.1,111.6,99.4,Midstream,FIXED,ALACN,67.0,2.0,CAD,ALA CN Equity,7.8,190,BBB-,>7,2.1-3.1,Non Financial Hybrids,VARIABLE,ALACN,212.4,3.3,CAD,ALA CN Equity,1.7,324,BB,1-2,>25.1,6.0,NBF,"3,000,000",16.0,CIBC,"2,000,000",1,0,571,649,02138ZAQ6,021361AC4
269,ALACN 4 ½ 08/15/44,ALACN 4.99 10/04/47,10.7,2.7,13.1,-12.6,-2.4,23.3,99.4,Midstream,FIXED,ALACN,179.0,19.1,CAD,ALA CN Equity,10.9,"1,077",BBB-,>7,10.1-25.1,Midstream,FIXED,ALACN,169.5,22.2,CAD,ALA CN Equity,7.8,"1,222",BBB-,>7,10.1-25.1,NaN,None,,NaN,SCM,"3,000,000",0,0,0,"3,667",02138ZAL7,02138ZAR4
280,ALACN 4.672 01/08/29,ALACN 8.9 11/10/2083,-134.3,2.7,-133.1,-244.1,-1.2,109.8,99.4,Midstream,FIXED,ALACN,79.4,3.4,CAD,ALA CN Equity,1.5,322,BBB-,1-2,3.1-4.1,Non Financial Hybrids,VARIABLE,ALACN,212.4,3.3,CAD,ALA CN Equity,1.7,324,BB,1-2,>25.1,5.0,SCM,"2,000,000",16.0,CIBC,"2,000,000",0,0,645,649,02138ZBH5,021361AC4
282,ALACN 5.16 01/13/44,ALACN 8.9 11/10/2083,-31.8,2.7,-29.8,-143.3,-2.0,111.5,99.4,Midstream,FIXED,ALACN,182.7,18.5,CAD,ALA CN Equity,11.5,"1,115",BBB-,>7,10.1-25.1,Non Financial Hybrids,VARIABLE,ALACN,212.4,3.3,CAD,ALA CN Equity,1.7,324,BB,1-2,>25.1,NaN,None,,16.0,CIBC,"2,000,000",0,0,0,649,02138ZAJ2,021361AC4
289,ALACN 2.166 03/16/27,ALACN 8.9 11/10/2083,-158.9,2.6,-156.5,-266.4,-2.5,107.4,99.4,Midstream,FIXED,ALACN,55.9,1.7,CAD,ALA CN Equity,4.3,159,BBB-,3-5,1-2.1,Non Financial Hybrids,VARIABLE,ALACN,212.4,3.3,CAD,ALA CN Equity,1.7,324,BB,1-2,>25.1,NaN,None,,16.0,CIBC,"2,000,000",0,0,0,649,02138ZAZ6,021361AC4


In [200]:
# Where CAD Bonds May Appear Cheap vs USD Bonds


z_cad_vs_usd = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'USD') &
    (z_all_enhanced['Custom_Sector_1'] == z_all_enhanced['Custom_Sector_2']) &
    (z_all_enhanced['Yrs (Mat) Bucket_1'] == z_all_enhanced['Yrs (Mat) Bucket_2']) &
    (z_all_enhanced['Ticker_1'] == z_all_enhanced['Ticker_2'])
].copy()

display(format_for_display(z_cad_vs_usd.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
9126,BAC 1.978 09/15/27,BAC 4.376 04/27/28,-1.1,0.2,11.4,-43.9,-12.4,42.9,51.4,Financial Maples,VARIABLE,BAC,53.1,1.2,CAD,BAC US Equity,4.1,113,A,3-5,2.1-3.1,Financial Maples,VARIABLE,BAC,83.6,1.8,USD,BAC US Equity,3.2,169,A,3-5,2.1-3.1,5.0,TD,"5,000,000",NaN,NaN,,0,0,563,0,060505FZ2,06051GKP3
9272,BAC 3.615 03/16/28,BAC 4.376 04/27/28,8.0,0.2,22.2,-23.0,-14.2,31.0,52.6,Financial Maples,VARIABLE,BAC,63.8,1.7,CAD,BAC US Equity,3.3,162,A,3-5,2.1-3.1,Financial Maples,VARIABLE,BAC,83.6,1.8,USD,BAC US Equity,3.2,169,A,3-5,2.1-3.1,3.0,NBF,"10,000,000",NaN,NaN,,0,0,"1,618",0,060505GF5,06051GKP3
10422,BAC 1.978 09/15/27,BAC 5.933 09/15/27,2.6,-0.1,23.0,-28.9,-20.4,31.5,43.1,Financial Maples,VARIABLE,BAC,53.1,1.2,CAD,BAC US Equity,4.1,113,A,3-5,2.1-3.1,Financial Maples,VARIABLE,BAC,71.1,1.2,USD,BAC US Equity,1.8,114,A,1-2,2.1-3.1,5.0,TD,"5,000,000",NaN,NaN,,0,0,563,0,060505FZ2,06051GLV9
10672,BAC 1.978 09/15/27,BAC 2.551 02/04/28,-4.6,-0.1,14.3,-39.7,-18.9,35.0,41.9,Financial Maples,VARIABLE,BAC,53.1,1.2,CAD,BAC US Equity,4.1,113,A,3-5,2.1-3.1,Financial Maples,VARIABLE,BAC,85.9,1.6,USD,BAC US Equity,3.4,145,A,3-5,2.1-3.1,5.0,TD,"5,000,000",NaN,NaN,,0,0,563,0,060505FZ2,06051GKJ7
10774,BAC 3.615 03/16/28,BAC 5.933 09/15/27,11.7,-0.2,33.8,-7.3,-22.1,19.0,37.9,Financial Maples,VARIABLE,BAC,63.8,1.7,CAD,BAC US Equity,3.3,162,A,3-5,2.1-3.1,Financial Maples,VARIABLE,BAC,71.1,1.2,USD,BAC US Equity,1.8,114,A,1-2,2.1-3.1,3.0,NBF,"10,000,000",NaN,NaN,,0,0,"1,618",0,060505GF5,06051GLV9
13056,ATH 5.113 03/07/29,ATH 5.583 01/09/29,-7.5,-0.6,23.8,-27.6,-31.3,20.1,37.5,Financial Maples,FIXED,ATH,90.1,3.6,CAD,ATH US Equity,1.4,346,A+,1-2,3.1-4.1,Financial Maples,FIXED,ATH,131.4,3.5,USD,ATH US Equity,1.5,321,A+,1-2,3.1-4.1,5.0,TD,"3,000,000",NaN,NaN,,0,0,"1,038",0,04686PAD9,04685A3Q2
13472,BAC 1.978 09/15/27,BAC 4.948 07/22/28,-8.9,-0.6,15.2,-39.9,-24.1,31.0,34.7,Financial Maples,VARIABLE,BAC,53.1,1.2,CAD,BAC US Equity,4.1,113,A,3-5,2.1-3.1,Financial Maples,VARIABLE,BAC,87.4,2.0,USD,BAC US Equity,3.0,192,A,2-3,2.1-3.1,5.0,TD,"5,000,000",NaN,NaN,,0,0,563,0,060505FZ2,06051GKW8
14377,BAC 3.615 03/16/28,BAC 4.948 07/22/28,0.2,-0.8,23.8,-23.1,-23.6,23.3,27.6,Financial Maples,VARIABLE,BAC,63.8,1.7,CAD,BAC US Equity,3.3,162,A,3-5,2.1-3.1,Financial Maples,VARIABLE,BAC,87.4,2.0,USD,BAC US Equity,3.0,192,A,2-3,2.1-3.1,3.0,NBF,"10,000,000",NaN,NaN,,0,0,"1,618",0,060505GF5,06051GKW8
14692,BAC 2.598 04/04/29,BAC 5.202 04/25/29,-7.6,-0.9,25.4,-23.2,-33.0,15.6,28.4,Financial Maples,VARIABLE,BAC,59.2,2.7,CAD,BAC US Equity,4.3,255,A,3-5,3.1-4.1,Financial Maples,VARIABLE,BAC,96.4,2.8,USD,BAC US Equity,2.2,261,A,2-3,3.1-4.1,6.0,TD,"3,000,000",NaN,NaN,,0,0,765,0,060505FU3,06051GLG2
16334,BCECN 4.05 03/17/51,BCECN 5.55 02/15/54,39.2,-1.3,79.9,31.0,-40.7,8.2,12.5,Cable/Telco,FIXED,BCECN,155.7,25.7,CAD,BCE CN Equity,4.3,"1,199",BBB,3-5,>25.1,Cable/Telco,FIXED,BCECN,205.5,28.6,USD,BCE CN Equity,1.4,"1,261",BBB,1-2,>25.1,10.0,RBC,"5,000,000",NaN,NaN,,0,0,"5,994",0,07813ZCH5,0778FPAN9


In [201]:
# Where CAD Bonds May Appear Rich vs USD Bonds


z_usd_vs_cad = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'USD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['Custom_Sector_1'] == z_all_enhanced['Custom_Sector_2']) &
    (z_all_enhanced['Yrs (Mat) Bucket_1'] == z_all_enhanced['Yrs (Mat) Bucket_2']) &
    (z_all_enhanced['Ticker_1'] == z_all_enhanced['Ticker_2'])
].copy()

display(format_for_display(z_usd_vs_cad.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
1477,BCECN 3.65 08/15/52,BCECN 5.6 08/11/53,-38.5,2.0,-37.8,-87.4,-0.7,48.8,99.4,Cable/Telco,FIXED,BCECN,201.7,27.1,USD,BCE CN Equity,3.4,995,BBB,3-5,>25.1,Cable/Telco,FIXED,BCECN,154.7,27.6,CAD,BCE CN Equity,1.9,"1,458",BBB,1-2,>25.1,NaN,NaN,,4.0,TD,"5,000,000",0,0,0,"7,289",0778FPAJ8,07813ZCP7
1560,BCECN 5.2 02/15/34,BCECN 5.85 11/10/32,-8.5,1.9,-7.2,-34.9,-1.3,26.4,98.6,Cable/Telco,FIXED,BCECN,158.5,8.6,USD,BCE CN Equity,1.4,677,BBB,1-2,7.1-10.1,Cable/Telco,FIXED,BCECN,112.6,7.1,CAD,BCE CN Equity,2.7,632,BBB,2-3,7.1-10.1,NaN,NaN,,4.0,NBF,"10,000,000",0,0,0,"6,319",0778FPAM1,07813ZCK8
1604,BCECN 3.65 08/15/52,BCECN 5.15 02/09/53,-38.5,1.9,-37.5,-88.7,-1.0,50.2,99.0,Cable/Telco,FIXED,BCECN,201.7,27.1,USD,BCE CN Equity,3.4,995,BBB,3-5,>25.1,Cable/Telco,FIXED,BCECN,154.6,27.6,CAD,BCE CN Equity,2.4,"1,395",BBB,2-3,>25.1,NaN,NaN,,3.0,TD,"5,000,000",0,0,0,"6,973",0778FPAJ8,07813ZCM4
2079,BCECN 3.65 03/17/51,BCECN 5.6 08/11/53,-47.6,1.8,-45.9,-84.6,-1.7,36.9,97.4,Cable/Telco,FIXED,BCECN,189.6,25.7,USD,BCE CN Equity,4.3,"1,007",BBB,3-5,>25.1,Cable/Telco,FIXED,BCECN,154.7,27.6,CAD,BCE CN Equity,1.9,"1,458",BBB,1-2,>25.1,NaN,NaN,,4.0,TD,"5,000,000",0,0,0,"7,289",0778FPAF6,07813ZCP7
2213,BCECN 3.65 03/17/51,BCECN 5.15 02/09/53,-47.6,1.7,-45.3,-85.9,-2.3,38.3,97.4,Cable/Telco,FIXED,BCECN,189.6,25.7,USD,BCE CN Equity,4.3,"1,007",BBB,3-5,>25.1,Cable/Telco,FIXED,BCECN,154.6,27.6,CAD,BCE CN Equity,2.4,"1,395",BBB,2-3,>25.1,NaN,NaN,,3.0,TD,"5,000,000",0,0,0,"6,973",0778FPAF6,07813ZCM4
2230,BCECN 3.2 02/15/52,BCECN 5.6 08/11/53,-50.0,1.7,-45.1,-87.9,-5.0,37.9,97.4,Cable/Telco,FIXED,BCECN,190.0,26.6,USD,BCE CN Equity,3.9,957,BBB,3-5,>25.1,Cable/Telco,FIXED,BCECN,154.7,27.6,CAD,BCE CN Equity,1.9,"1,458",BBB,1-2,>25.1,NaN,NaN,,4.0,TD,"5,000,000",0,0,0,"7,289",0778FPAH2,07813ZCP7
2316,BCECN 3.2 02/15/52,BCECN 5.15 02/09/53,-50.0,1.7,-44.0,-89.8,-6.0,39.8,97.8,Cable/Telco,FIXED,BCECN,190.0,26.6,USD,BCE CN Equity,3.9,957,BBB,3-5,>25.1,Cable/Telco,FIXED,BCECN,154.6,27.6,CAD,BCE CN Equity,2.4,"1,395",BBB,2-3,>25.1,NaN,NaN,,3.0,TD,"5,000,000",0,0,0,"6,973",0778FPAH2,07813ZCM4
2396,BCECN 5.55 02/15/54,BCECN 5.6 08/11/53,-38.4,1.7,-37.7,-80.1,-0.8,41.6,98.6,Cable/Telco,FIXED,BCECN,205.5,28.6,USD,BCE CN Equity,1.4,"1,261",BBB,1-2,>25.1,Cable/Telco,FIXED,BCECN,154.7,27.6,CAD,BCE CN Equity,1.9,"1,458",BBB,1-2,>25.1,NaN,NaN,,4.0,TD,"5,000,000",0,0,0,"7,289",0778FPAN9,07813ZCP7
2465,BCECN 4.3 07/29/49,BCECN 4.35 12/18/45,-36.8,1.7,-35.1,-84.3,-1.7,47.5,98.2,Cable/Telco,FIXED,BCECN,207.4,24.0,USD,BCE CN Equity,6.2,"1,030",BBB,5-7,10.1-25.1,Cable/Telco,FIXED,BCECN,162.2,20.4,CAD,BCE CN Equity,10.3,"1,124",BBB,>7,10.1-25.1,NaN,NaN,,5.0,TD,"3,000,000",0,0,0,"3,372",0778FPAB5,07813ZBR4
2698,BCECN 4.3 07/29/49,BCECN 4.45 02/27/47,-35.7,1.6,-34.2,-84.1,-1.5,48.4,97.0,Cable/Telco,FIXED,BCECN,207.4,24.0,USD,BCE CN Equity,6.2,"1,030",BBB,5-7,10.1-25.1,Cable/Telco,FIXED,BCECN,160.0,21.6,CAD,BCE CN Equity,8.4,"1,165",BBB,>7,10.1-25.1,NaN,NaN,,5.0,TD,"5,000,000",0,0,0,"5,826",0778FPAB5,07813ZBX1


In [202]:
# Which Bonds in Our Portfolio May Look Rich vs Universe

z_port_vs_universe = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['Own_CUSIP_2'] == 1)
].copy()

display(format_for_display(z_port_vs_universe.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
1947,BAMREP 3.63 01/15/27,BIP 5.71 07/27/30,-35.4,1.8,-33.2,-57.5,-2.2,22.1,97.8,Power Gen/Renewable,FIXED,BAMREP,63.8,1.3,CAD,BEP US Equity,8.9,122,BBB+,>7,1-2.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,5.0,TD,"5,000,000",6.0,RBC,"5,000,000",0,1,608,"2,282",11282ZAL2,11291ZAM9
2274,BCRES 0 06/18/43,BIP 5.71 07/27/30,5.2,1.7,6.7,-18.7,-1.5,23.9,96.2,CAD Govt,ZERO COUPON,BCRES,103.5,17.9,CAD,nan,22.1,755,NR,>7,10.1-25.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,NaN,NaN,,6.0,RBC,"5,000,000",0,1,0,"2,282",11066ZWM8,11291ZAM9
2381,ALTALK 4.872 11/15/40,BIP 5.71 07/27/30,25.3,1.7,25.3,8.3,0.0,17.0,99.8,Utility,FIXED,ALTALK,121.4,15.3,CAD,BRK/A US Equity,14.7,"1,067",A-,>7,10.1-25.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,2.0,NBF,"2,000,000",6.0,RBC,"5,000,000",0,1,"2,134","2,282",02135ZAD8,11291ZAM9
2462,BCECN 3.55 03/02/26,BIP 5.71 07/27/30,-50.6,1.7,-47.8,-70.2,-2.8,19.6,98.2,Cable/Telco,FIXED,BCECN,49.2,0.4,CAD,BCE CN Equity,9.4,38,BBB,>7,0.50-1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,5.0,SCM,"10,171,000",6.0,RBC,"5,000,000",0,1,386,"2,282",07813ZBT0,11291ZAM9
2473,BCECN 2.9 08/12/26,BIP 5.71 07/27/30,-48.7,1.7,-46.7,-66.2,-2.0,17.5,92.7,Cable/Telco,FIXED,BCECN,48.5,1.1,CAD,BCE CN Equity,8.9,105,BBB,>7,1-2.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,4.0,TD,"5,000,000",6.0,RBC,"5,000,000",0,1,524,"2,282",07813ZBV5,11291ZAM9
2485,AQNCN 4.09 02/17/27,BIP 5.71 07/27/30,-35.4,1.7,-34.2,-71.8,-1.3,36.4,97.8,Power Gen/Renewable,FIXED,AQNCN,NaN,2.0,CAD,AQN CN Equity,8.5,,NR,>7,1-2.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,NaN,NaN,,6.0,RBC,"5,000,000",0,1,0,"2,282",01585PAG0,11291ZAM9
2928,ACICN 3.15 04/06/26,BIP 5.71 07/27/30,-38.2,1.5,-34.0,-54.8,-4.2,16.7,93.8,Utility,FIXED,ACICN,60.5,0.7,CAD,ACI CN Equity,6.3,71,BBB+,5-7,0.50-1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,NaN,None,,6.0,RBC,"5,000,000",0,1,0,"2,282",02091ZAB4,11291ZAM9
3061,AQNCN 4.6 01/29/29,BIP 5.71 07/27/30,-27.3,1.5,-26.0,-63.7,-1.3,36.4,97.8,Power Gen/Renewable,FIXED,AQNCN,NaN,4.0,CAD,AQN CN Equity,6.5,,NR,5-7,3.1-4.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,NaN,NaN,,6.0,RBC,"5,000,000",0,1,0,"2,282",01585PAJ4,11291ZAM9
3083,ALTALK 2.747 05/29/26,BIP 5.71 07/27/30,-74.4,1.5,-72.3,-99.2,-2.1,24.8,97.0,Utility,FIXED,ALTALK,24.0,0.9,CAD,BRK/A US Equity,9.1,85,A-,>7,0.50-1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,7.0,RBC,"10,000,000",6.0,RBC,"5,000,000",0,1,854,"2,282",02135ZAR7,11291ZAM9
3124,ALTALK 4.462 11/08/41,BIP 5.71 07/27/30,23.0,1.5,23.0,6.9,0.0,16.1,99.8,Utility,FIXED,ALTALK,119.3,16.3,CAD,BRK/A US Equity,13.7,"1,080",A-,>7,10.1-25.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,1.0,NBF,"2,000,000",6.0,RBC,"5,000,000",0,1,"2,159","2,282",02135ZAE6,11291ZAM9


In [203]:
# Which Bonds in Our Portfolio May Look Rich vs Universe & We Can Execute

z_port_vs_universe_2way_liquidity = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['CUSIP_1 CR01 @ Wide Offer'] >2000) &
    (z_all_enhanced['CUSIP_2 CR01 @ Best Bid'] >2000) &
    (z_all_enhanced['Own_CUSIP_2'] == 1)
].copy()

display(format_for_display(z_port_vs_universe_2way_liquidity.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
2381,ALTALK 4.872 11/15/40,BIP 5.71 07/27/30,25.3,1.7,25.3,8.3,0.0,17.0,99.8,Utility,FIXED,ALTALK,121.4,15.3,CAD,BRK/A US Equity,14.7,"1,067",A-,>7,10.1-25.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,2.0,NBF,"2,000,000",6.0,RBC,"5,000,000",0,1,"2,134","2,282",02135ZAD8,11291ZAM9
3124,ALTALK 4.462 11/08/41,BIP 5.71 07/27/30,23.0,1.5,23.0,6.9,0.0,16.1,99.8,Utility,FIXED,ALTALK,119.3,16.3,CAD,BRK/A US Equity,13.7,"1,080",A-,>7,10.1-25.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,1.0,NBF,"2,000,000",6.0,RBC,"5,000,000",0,1,"2,159","2,282",02135ZAE6,11291ZAM9
3976,BCFERR 4.289 04/28/44,BIP 5.71 07/27/30,19.8,1.3,21.3,1.8,-1.5,18.0,92.7,Infastructure,FIXED,BCFERR,118.3,18.8,CAD,1559415D CN Equity,11.2,"1,155",A+,>7,10.1-25.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,7.0,RBC,"5,000,000",6.0,RBC,"5,000,000",0,1,"5,773","2,282",110574AJ9,11291ZAM9
5589,ALTALK 4.054 11/21/44,BIP 5.71 07/27/30,18.2,0.9,22.3,2.3,-4.2,15.9,83.5,Utility,FIXED,ALTALK,114.7,19.4,CAD,BRK/A US Equity,10.6,"1,156",A-,>7,10.1-25.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,1.0,NBF,"3,000,000",6.0,RBC,"5,000,000",0,1,"3,468","2,282",02135ZAP1,11291ZAM9
5944,ALCTRA 4.627 06/13/34,BIP 5.71 07/27/30,-15.8,0.8,-13.3,-31.6,-2.5,15.9,75.6,Utility,FIXED,ALCTRA,82.5,8.7,CAD,1487407D CN Equity,1.1,730,A-,1-2,7.1-10.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,5.0,RBC,"3,000,000",6.0,RBC,"5,000,000",0,1,"2,189","2,282",014443AN5,11291ZAM9
6154,ALTALK 5.381 03/26/40,BIP 5.71 07/27/30,24.3,0.8,41.8,7.8,-17.5,16.5,84.3,Utility,FIXED,ALTALK,122.1,14.7,CAD,BRK/A US Equity,15.3,"1,072",A-,>7,10.1-25.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,-1.0,BMO,"2,000,000",6.0,RBC,"5,000,000",0,1,"2,143","2,282",02135ZAC0,11291ZAM9
6163,AERMON 3.919 09/26/42,BIP 5.71 07/27/30,15.2,0.8,20.0,-0.5,-4.8,15.8,83.5,Airport,FIXED,AERMON,114.2,17.2,CAD,58232Z CN Equity,12.8,"1,076",A+,>7,10.1-25.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,5.0,NBF,"3,000,000",6.0,RBC,"5,000,000",0,1,"3,227","2,282",007863AH6,11291ZAM9
6489,ALTALK 3.99 06/30/42,BIP 5.71 07/27/30,19.3,0.7,23.1,5.5,-3.8,13.8,75.6,Utility,FIXED,ALTALK,117.9,17.0,CAD,BRK/A US Equity,13.0,"1,066",A-,>7,10.1-25.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,4.0,NBF,"2,000,000",6.0,RBC,"5,000,000",0,1,"2,132","2,282",02135ZAF3,11291ZAM9
6861,ALTALK 4.922 09/17/43,BIP 5.71 07/27/30,18.1,0.7,21.8,3.7,-3.8,14.4,68.1,Utility,FIXED,ALTALK,116.7,18.2,CAD,BRK/A US Equity,11.8,"1,196",A-,>7,10.1-25.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,5.0,TD,"2,000,000",6.0,RBC,"5,000,000",0,1,"2,392","2,282",02135ZAK2,11291ZAM9
7004,BCFERR 2.794 10/15/49,BIP 5.71 07/27/30,12.1,0.6,16.8,-6.4,-4.7,18.4,70.8,Infastructure,FIXED,BCFERR,110.3,24.3,CAD,1559415D CN Equity,5.7,"1,113",A+,5-7,10.1-25.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,7.0,RBC,"3,000,000",6.0,RBC,"5,000,000",0,1,"3,339","2,282",110574AK6,11291ZAM9


In [204]:
# Which Bonds in Our Portfolio May Look Rich vs Universe & We Can Execute At Least 1 Side

z_port_vs_universe_1way_liquidity = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['CUSIP_1 CR01 @ Wide Offer'] >2000) &
    (z_all_enhanced['Own_CUSIP_2'] == 1)
].copy()

display(format_for_display(z_port_vs_universe_1way_liquidity.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
2381,ALTALK 4.872 11/15/40,BIP 5.71 07/27/30,25.3,1.7,25.3,8.3,0.0,17.0,99.8,Utility,FIXED,ALTALK,121.4,15.3,CAD,BRK/A US Equity,14.7,"1,067",A-,>7,10.1-25.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,2.0,NBF,"2,000,000",6.0,RBC,"5,000,000",0,1,"2,134","2,282",02135ZAD8,11291ZAM9
3124,ALTALK 4.462 11/08/41,BIP 5.71 07/27/30,23.0,1.5,23.0,6.9,0.0,16.1,99.8,Utility,FIXED,ALTALK,119.3,16.3,CAD,BRK/A US Equity,13.7,"1,080",A-,>7,10.1-25.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,1.0,NBF,"2,000,000",6.0,RBC,"5,000,000",0,1,"2,159","2,282",02135ZAE6,11291ZAM9
3976,BCFERR 4.289 04/28/44,BIP 5.71 07/27/30,19.8,1.3,21.3,1.8,-1.5,18.0,92.7,Infastructure,FIXED,BCFERR,118.3,18.8,CAD,1559415D CN Equity,11.2,"1,155",A+,>7,10.1-25.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,7.0,RBC,"5,000,000",6.0,RBC,"5,000,000",0,1,"5,773","2,282",110574AJ9,11291ZAM9
5589,ALTALK 4.054 11/21/44,BIP 5.71 07/27/30,18.2,0.9,22.3,2.3,-4.2,15.9,83.5,Utility,FIXED,ALTALK,114.7,19.4,CAD,BRK/A US Equity,10.6,"1,156",A-,>7,10.1-25.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,1.0,NBF,"3,000,000",6.0,RBC,"5,000,000",0,1,"3,468","2,282",02135ZAP1,11291ZAM9
5944,ALCTRA 4.627 06/13/34,BIP 5.71 07/27/30,-15.8,0.8,-13.3,-31.6,-2.5,15.9,75.6,Utility,FIXED,ALCTRA,82.5,8.7,CAD,1487407D CN Equity,1.1,730,A-,1-2,7.1-10.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,5.0,RBC,"3,000,000",6.0,RBC,"5,000,000",0,1,"2,189","2,282",014443AN5,11291ZAM9
6154,ALTALK 5.381 03/26/40,BIP 5.71 07/27/30,24.3,0.8,41.8,7.8,-17.5,16.5,84.3,Utility,FIXED,ALTALK,122.1,14.7,CAD,BRK/A US Equity,15.3,"1,072",A-,>7,10.1-25.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,-1.0,BMO,"2,000,000",6.0,RBC,"5,000,000",0,1,"2,143","2,282",02135ZAC0,11291ZAM9
6163,AERMON 3.919 09/26/42,BIP 5.71 07/27/30,15.2,0.8,20.0,-0.5,-4.8,15.8,83.5,Airport,FIXED,AERMON,114.2,17.2,CAD,58232Z CN Equity,12.8,"1,076",A+,>7,10.1-25.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,5.0,NBF,"3,000,000",6.0,RBC,"5,000,000",0,1,"3,227","2,282",007863AH6,11291ZAM9
6489,ALTALK 3.99 06/30/42,BIP 5.71 07/27/30,19.3,0.7,23.1,5.5,-3.8,13.8,75.6,Utility,FIXED,ALTALK,117.9,17.0,CAD,BRK/A US Equity,13.0,"1,066",A-,>7,10.1-25.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,4.0,NBF,"2,000,000",6.0,RBC,"5,000,000",0,1,"2,132","2,282",02135ZAF3,11291ZAM9
6777,AERMON 3.919 09/26/42,ALACN 3.98 10/04/27,46.1,0.7,52.5,27.6,-6.4,18.5,76.4,Airport,FIXED,AERMON,114.2,17.2,CAD,58232Z CN Equity,12.8,"1,076",A+,>7,10.1-25.1,Midstream,FIXED,ALACN,67.0,2.0,CAD,ALA CN Equity,7.8,190,BBB-,>7,2.1-3.1,5.0,NBF,"3,000,000",6.0,NBF,"5,000,000",0,1,"3,227",951,007863AH6,02138ZAQ6
6861,ALTALK 4.922 09/17/43,BIP 5.71 07/27/30,18.1,0.7,21.8,3.7,-3.8,14.4,68.1,Utility,FIXED,ALTALK,116.7,18.2,CAD,BRK/A US Equity,11.8,"1,196",A-,>7,10.1-25.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,5.0,TD,"2,000,000",6.0,RBC,"5,000,000",0,1,"2,392","2,282",02135ZAK2,11291ZAM9


In [205]:
# Which Bonds in Our Portfolio May Look Rich vs Universe & We Can Execute Both Sides At Reasonable Bid/Offer

z_port_vs_universe_2way_liquiditybidoffer = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['CUSIP_1 CR01 @ Wide Offer'] >2000) &
    (z_all_enhanced['CUSIP_2 CR01 @ Best Bid'] >2000) &
    (z_all_enhanced['Bid/Offer_1'] <2) &
    (z_all_enhanced['Bid/Offer_2'] <2) &
    (z_all_enhanced['Own_CUSIP_2'] == 1)
].copy()

display(format_for_display(z_port_vs_universe_2way_liquiditybidoffer.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2


In [206]:
# Which Bonds in Our Portfolio May Look Rich vs Universe & We Can Execute At Least One Side At Reasonable Bid/Offer

z_port_vs_universe_1way_liquiditybidoffer = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['CUSIP_1 CR01 @ Wide Offer'] >2000) &
    (z_all_enhanced['Bid/Offer_1'] <2) &
    (z_all_enhanced['Own_CUSIP_2'] == 1)
].copy()

display(format_for_display(z_port_vs_universe_1way_liquiditybidoffer.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
3124,ALTALK 4.462 11/08/41,BIP 5.71 07/27/30,23.0,1.5,23.0,6.9,0.0,16.1,99.8,Utility,FIXED,ALTALK,119.3,16.3,CAD,BRK/A US Equity,13.7,"1,080",A-,>7,10.1-25.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,1.0,NBF,"2,000,000",6.0,RBC,"5,000,000",0,1,"2,159","2,282",02135ZAE6,11291ZAM9
5589,ALTALK 4.054 11/21/44,BIP 5.71 07/27/30,18.2,0.9,22.3,2.3,-4.2,15.9,83.5,Utility,FIXED,ALTALK,114.7,19.4,CAD,BRK/A US Equity,10.6,"1,156",A-,>7,10.1-25.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,1.0,NBF,"3,000,000",6.0,RBC,"5,000,000",0,1,"3,468","2,282",02135ZAP1,11291ZAM9
6154,ALTALK 5.381 03/26/40,BIP 5.71 07/27/30,24.3,0.8,41.8,7.8,-17.5,16.5,84.3,Utility,FIXED,ALTALK,122.1,14.7,CAD,BRK/A US Equity,15.3,"1,072",A-,>7,10.1-25.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,-1.0,BMO,"2,000,000",6.0,RBC,"5,000,000",0,1,"2,143","2,282",02135ZAC0,11291ZAM9
7364,ALTALK 4.09 06/30/45,BIP 5.71 07/27/30,16.1,0.6,19.8,1.5,-3.7,14.6,67.7,Utility,FIXED,ALTALK,114.4,20.0,CAD,BRK/A US Equity,10.0,"1,179",A-,>7,10.1-25.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,1.0,NBF,"5,000,000",6.0,RBC,"5,000,000",0,1,"5,894","2,282",02135ZAQ9,11291ZAM9
11523,ALCTRA 5.225 11/14/52,BIP 5.71 07/27/30,6.8,-0.3,17.0,-6.7,-10.2,13.5,36.7,Utility,FIXED,ALCTRA,104.5,26.8,CAD,1487407D CN Equity,2.7,"1,539",A-,2-3,>25.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,1.0,BMO,"2,000,000",6.0,RBC,"5,000,000",0,1,"3,077","2,282",014443AM7,11291ZAM9
13177,ALACN 2.477 11/30/30,ALACN 3.98 10/04/27,23.5,-0.6,38.9,18.5,-15.4,4.9,26.4,Midstream,FIXED,ALACN,87.7,5.4,CAD,ALA CN Equity,4.6,459,BBB-,3-5,5.1-7.1,Midstream,FIXED,ALACN,67.0,2.0,CAD,ALA CN Equity,7.8,190,BBB-,>7,2.1-3.1,-2.0,BMO,"5,000,000",6.0,NBF,"5,000,000",0,1,"2,293",951,02138ZAX1,02138ZAQ6
13587,ALACN 2.477 11/30/30,BIP 5.71 07/27/30,-7.4,-0.6,3.3,-12.7,-10.7,5.3,31.2,Midstream,FIXED,ALACN,87.7,5.4,CAD,ALA CN Equity,4.6,459,BBB-,3-5,5.1-7.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,-2.0,BMO,"5,000,000",6.0,RBC,"5,000,000",0,1,"2,293","2,282",02138ZAX1,11291ZAM9
14696,ALACN 2.477 11/30/30,ALACN 4.638 05/15/26,36.5,-0.9,78.7,25.4,-42.2,11.1,11.7,Midstream,FIXED,ALACN,87.7,5.4,CAD,ALA CN Equity,4.6,459,BBB-,3-5,5.1-7.1,Midstream,FIXED,ALACN,50.6,0.8,CAD,ALA CN Equity,2.2,83,BBB-,2-3,0.50-1,-2.0,BMO,"5,000,000",10.0,NBF,"10,000,000",0,1,"2,293",825,02138ZAX1,02138ZBG7
15292,BCECN 2 ½ 05/14/30,BIP 5.71 07/27/30,-25.1,-1.0,-3.1,-32.2,-21.9,7.1,22.4,Cable/Telco,FIXED,BCECN,73.1,4.8,CAD,BCE CN Equity,5.2,423,BBB,5-7,4.1-5.1,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,-4.0,BMO,"5,000,000",6.0,RBC,"5,000,000",0,1,"2,117","2,282",07813ZCE2,11291ZAM9
17657,ALACN 2.477 11/30/30,ALACN 4.12 04/07/26,34.7,-1.7,54.3,31.4,-19.6,3.3,4.2,Midstream,FIXED,ALACN,87.7,5.4,CAD,ALA CN Equity,4.6,459,BBB-,3-5,5.1-7.1,Midstream,FIXED,ALACN,56.3,0.5,CAD,ALA CN Equity,9.3,48,BBB-,>7,0.50-1,-2.0,BMO,"5,000,000",5.0,NBF,"10,000,000",0,1,"2,293",477,02138ZAX1,02138ZAP8


In [207]:
# Bonds That May Look Cheap and We Can Execute At Least One Side At Reasonable Bid/Offer

z_cad_universe_cr01_bidoffer = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['CUSIP_1 CR01 @ Wide Offer'] >2000) &
    (z_all_enhanced['Bid/Offer_1'] <1) 
].copy()

display(format_for_display(z_cad_universe_cr01_bidoffer.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
233,ALACN 2.477 11/30/30,ALACN 8.9 11/10/2083,-123.1,2.8,-123.1,-229.0,0.0,106.0,99.8,Midstream,FIXED,ALACN,87.7,5.4,CAD,ALA CN Equity,4.6,459,BBB-,3-5,5.1-7.1,Non Financial Hybrids,VARIABLE,ALACN,212.4,3.3,CAD,ALA CN Equity,1.7,324,BB,1-2,>25.1,-2.0,BMO,"5,000,000",16.0,CIBC,"2,000,000",0,0,"2,293",649,02138ZAX1,021361AC4
235,ALTALK 5.381 03/26/40,ATIMLP 6.069 02/05/42,-32.8,2.8,-30.7,-86.5,-2.0,53.7,99.0,Utility,FIXED,ALTALK,122.1,14.7,CAD,BRK/A US Equity,15.3,"1,072",A-,>7,10.1-25.1,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,-1.0,BMO,"2,000,000",NaN,NaN,,0,0,"2,143",0,02135ZAC0,04682BAA0
348,ALTALK 5.381 03/26/40,TVECN 7 ¼ 05/10/27,-112.8,2.5,-112.0,-314.8,-0.8,202.0,99.4,Utility,FIXED,ALTALK,122.1,14.7,CAD,BRK/A US Equity,15.3,"1,072",A-,>7,10.1-25.1,HY,FIXED,TVECN,234.1,0.8,CAD,TVE CN Equity,2.8,81,nan,2-3,1-2.1,-1.0,BMO,"2,000,000",NaN,NaN,,0,0,"2,143",0,02135ZAC0,87505YAD4
355,ALTALK 5.381 03/26/40,BCIMCR 2.551 06/24/26,106.6,2.5,110.2,79.6,-3.6,27.0,96.6,Utility,FIXED,ALTALK,122.1,14.7,CAD,BRK/A US Equity,15.3,"1,072",A-,>7,10.1-25.1,Pension,FIXED,BCIMCR,16.0,0.9,CAD,1790650D CN Equity,3.4,92,NR,3-5,0.50-1,-1.0,BMO,"2,000,000",NaN,CIBC,"10,000,000",0,0,"2,143",921,02135ZAC0,05554PAE3
371,ALACN 2.477 11/30/30,ALACN 7.35 08/17/2082,-102.5,2.5,-102.5,-226.8,0.0,124.2,99.8,Midstream,FIXED,ALACN,87.7,5.4,CAD,ALA CN Equity,4.6,459,BBB-,3-5,5.1-7.1,Non Financial Hybrids,VARIABLE,ALACN,192.6,2.1,CAD,ALA CN Equity,2.9,204,BB,2-3,>25.1,-2.0,BMO,"5,000,000",24.0,CIBC,"2,000,000",0,0,"2,293",408,02138ZAX1,021361AB6
421,ALACN 2.477 11/30/30,TVECN 7 ¼ 05/10/27,-144.5,2.4,-144.5,-332.4,0.0,187.9,99.8,Midstream,FIXED,ALACN,87.7,5.4,CAD,ALA CN Equity,4.6,459,BBB-,3-5,5.1-7.1,HY,FIXED,TVECN,234.1,0.8,CAD,TVE CN Equity,2.8,81,nan,2-3,1-2.1,-2.0,BMO,"5,000,000",NaN,NaN,,0,0,"2,293",0,02138ZAX1,87505YAD4
516,ALACN 2.477 11/30/30,BCIMCR 2.551 06/24/26,74.9,2.3,74.9,54.2,0.0,20.6,99.8,Midstream,FIXED,ALACN,87.7,5.4,CAD,ALA CN Equity,4.6,459,BBB-,3-5,5.1-7.1,Pension,FIXED,BCIMCR,16.0,0.9,CAD,1790650D CN Equity,3.4,92,NR,3-5,0.50-1,-2.0,BMO,"5,000,000",NaN,CIBC,"10,000,000",0,0,"2,293",921,02138ZAX1,05554PAE3
571,BIP 5.95 07/27/53,TVECN 7 ¼ 05/10/27,-71.5,2.3,-68.7,-257.4,-2.7,186.0,99.4,Infastructure,FIXED,BIP,165.4,27.5,CAD,BIP US Equity,2.0,"1,472",BBB+,1-2,>25.1,HY,FIXED,TVECN,234.1,0.8,CAD,TVE CN Equity,2.8,81,nan,2-3,1-2.1,0.0,BMO,"2,000,000",NaN,NaN,,0,0,"2,945",0,11291ZAN7,87505YAD4
621,ALACN 2.477 11/30/30,ATIMLP 6.069 02/05/42,-64.5,2.2,-46.9,-115.3,-17.6,50.8,98.6,Midstream,FIXED,ALACN,87.7,5.4,CAD,ALA CN Equity,4.6,459,BBB-,3-5,5.1-7.1,Other,FIXED,ATIMLP,153.3,10.1,CAD,2171732D CN Equity,2.8,815,NR,2-3,10.1-25.1,-2.0,BMO,"5,000,000",NaN,NaN,,0,0,"2,293",0,02138ZAX1,04682BAA0
787,ALTALK 5.381 03/26/40,BMO 1.928 07/22/31,90.7,2.2,95.0,59.9,-4.3,30.8,96.2,Utility,FIXED,ALTALK,122.1,14.7,CAD,BRK/A US Equity,15.3,"1,072",A-,>7,10.1-25.1,NVCC Sub Debt,VARIABLE,BMO,32.7,1.0,CAD,BMO CN Equity,4.0,99,BBB+,3-5,5.1-7.1,-1.0,BMO,"2,000,000",4.0,CIBC,"20,000,000",0,0,"2,143","1,976",02135ZAC0,06369ZCG7
